<a href="https://colab.research.google.com/github/WelfLowe/RLAgents/blob/main/knapsack_with_DP_and_Q_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Knapsack problem with Dynamic Programming and Reinforcement Learning

In the simple knapsack problem, the goal is to maximize the value of items that can fit in a knapsack with a given weight capacity.

We can solve this problem using:

1. Dynamic Programming (DP): A classic approach to solve the knapsack problem.
1. Reinforcement Learning (RL): Using Q-learning to find an optimal policy for item selection.

**Dynamic Programming (DP)**:

* Efficient and guarantees the optimal solution.
* Requires explicit problem formulation with well-defined states and transitions.

**Reinforcement Learning (RL)**:

* Simulates the problem iteratively and learns through exploration and exploitation.
* May not guarantee the optimal solution but is adaptable to problems with more complex or uncertain environments.

Both methods should yield similar results for this simple problem. The RL approach, however, may show slight variations due to its stochastic nature.

## Problem Description
We have a set of items, each with a weight and value. The goal is to select a subset of items such that:

The total weight does not exceed the knapsack's capacity.

The total value is maximized.


## Example
Items:
* Item 0: Weight = 1, Value = 1
* Item 1: Weight = 2, Value = 6
* Item 2: Weight = 3, Value = 10
* Item 3: Weight = 2, Value = 7

Knapsack Capacity: 5

Maximum value: 17 selecting items 2 and 3.

## 1. Solution with **dynamic programming**

In [1]:
def knapsack_dp(values, weights, capacity):
    n = len(values)
    dp = [[0] * (capacity + 1) for _ in range(n + 1)]

    for i in range(1, n + 1):
        for w in range(capacity + 1):
            if weights[i - 1] <= w:
                dp[i][w] = max(dp[i - 1][w], dp[i - 1][w - weights[i - 1]] + values[i - 1])
            else:
                dp[i][w] = dp[i - 1][w]

    # Reconstruct the solution
    w = capacity
    selected_items = []
    for i in range(n, 0, -1):
        if dp[i][w] != dp[i - 1][w]:
            selected_items.append(i - 1)
            w -= weights[i - 1]

    return dp[n][capacity], selected_items


# Problem definition
values = [1, 6, 10, 7]
weights = [1, 2, 3, 2]
capacity = 5

max_value, items = knapsack_dp(values, weights, capacity)
print(f"Maximum value: {max_value}")
print(f"Selected items: {items}")


Maximum value: 17
Selected items: [3, 2]


## 2. Solution with **Q-learning**

In this approach:

* States represent the current capacity and the index of the item being considered.
* Actions represent whether to include the current item in the knapsack or not.

In [2]:
import numpy as np

def knapsack_rl(values, weights, capacity, episodes=1000, alpha=0.1, gamma=0.9, epsilon=1.0, epsilon_decay=0.99):
    n = len(values)
    q_table = np.zeros((capacity + 1, n + 1, 2))  # (remaining capacity, item index, action)

    for episode in range(episodes):
        state = (capacity, 0)
        total_reward = 0
        done = False

        while not done:
            cap, item = state

            # Choose action (epsilon-greedy)
            if np.random.rand() < epsilon:
                action = np.random.choice([0, 1])  # 0: skip, 1: take
            else:
                action = np.argmax(q_table[cap, item])

            # Take action
            if action == 1 and item < n and weights[item] <= cap:
                next_state = (cap - weights[item], item + 1)
                reward = values[item]
            else:
                next_state = (cap, item + 1)
                reward = 0

            if next_state[1] >= n:  # End of items
                done = True

            # Update Q-value
            next_q = 0 if done else np.max(q_table[next_state[0], next_state[1]])
            q_table[cap, item, action] += alpha * (reward + gamma * next_q - q_table[cap, item, action])

            state = next_state
            total_reward += reward

        # Decay epsilon
        epsilon = max(0.1, epsilon * epsilon_decay)

    # Reconstruct the solution
    state = (capacity, 0)
    selected_items = []
    while state[1] < n:
        cap, item = state
        action = np.argmax(q_table[cap, item])
        if action == 1 and weights[item] <= cap:
            selected_items.append(item)
            state = (cap - weights[item], item + 1)
        else:
            state = (cap, item + 1)

    return sum(values[i] for i in selected_items), selected_items


# Problem definition
values = [1, 6, 10, 7]
weights = [1, 2, 3, 2]
capacity = 5

max_value, items = knapsack_rl(values, weights, capacity)
print(f"Maximum value: {max_value}")
print(f"Selected items: {items}")


Maximum value: 16
Selected items: [1, 2]


## Taks:
1. Understand the example
1. Visualize the learning history of Q-learning
1. Improve RL so that it arrives at the same result as DP in the simple example
1. Increase the problem size and compare RL and DL